In [2]:
from dotenv import load_dotenv
import pandas as pd
from datetime import date, datetime
from src.model.repositorios.xls.movimentacao_estoque import MovimentacaoDeEstoqueXls

In [ ]:
load_dotenv()

In [4]:
mov_estoque = MovimentacaoDeEstoqueXls()

In [5]:
dt_inicial = datetime(2024, 1, 1)
dt_final = datetime(2024, 8, 30)

mov = mov_estoque.buscar_movimentacao(dt_inicial=dt_inicial, dt_final=dt_final)

In [ ]:
df = pd.read_csv('data.csv')

df['descricao'] = df['descricao'].str.capitalize()
df['marca'] = df['marca'].str.capitalize()
df['data_movimento'] = pd.to_datetime(df['data_movimento'], format='%d/%m/%Y %H:%M:%S')
df['valor_total'] = df['qtd_negociada'] * (df['vlr_unitario'] - df['vlr_desconto'])

df

In [6]:
columns_names: set = {
        'nro_documento',
        'tipo_movimento',
        'cod_parceiro',
        'data_movimento',
        'cod_produto',
        'descricao',
        'marca',
        'qtd_negociada',
        'vlr_unitario',
        'vlr_desconto'
    }

In [ ]:
{item: item in df.columns for item in columns_names}

In [ ]:
df[(df['tipo_movimento'] == 'V') & (df['data_movimento'].between(datetime(2024,9,14), datetime(2024,9,16), inclusive='neither'))].sort_values('data_movimento', ascending=False)

In [ ]:
#df_marca = df[df['tipo_movimento'] == 'V'].groupby(['marca'])[['qtd_negociada', 'vlr_unitario']].sum().reset_index()

df_marca = df[(df['tipo_movimento'] == 'V') & (df['data_movimento'].between(datetime(2024,9,15), datetime(2024,9,17)))]\
    .groupby(['marca'])\
    .agg({'nro_documento': 'nunique', 'qtd_negociada': 'sum', 'vlr_unitario': 'sum'})\
    .sort_values('vlr_unitario', ascending=False)\
    .reset_index()

df_marca['vlr_unitario'] = df_marca['vlr_unitario'].apply(lambda x: round(x, 2))
df_marca

### Produtos

In [ ]:
df_produtos = df[(df['tipo_movimento'] == 'V') & (df['data_movimento'].between(datetime(2024,9,1), datetime(2024,9,15)))]\
    .groupby(['cod_produto', 'descricao', 'marca'])\
    .agg({'nro_documento': 'nunique', 'qtd_negociada': 'sum', 'valor_total': 'sum'})\
    .sort_values('valor_total', ascending=False)\
    .reset_index()

df_produtos['perc_vendas'] = df_produtos[['valor_total']] / df_produtos[['valor_total']].sum()

total = round(df_produtos[['valor_total']].sum(), 2)
df_produtos['valor_total_acu'] = round(df_produtos['valor_total'].cumsum(), 2)

df_produtos['perc_vendas_acu'] = df_produtos[['perc_vendas']].cumsum()

df_produtos['curva_abc'] = ''

df_produtos